In [1]:
!pip3 install --quiet --upgrade git+https://github.com/hpe-container-platform-community/hpecp-client@master
!pip3 install --quiet kubernetes

In [3]:
import os
try:
    user_paths = os.environ['PYTHONPATH'].split(os.pathsep)
except KeyError:
    user_paths = []
    
print(user_paths)

['.', '']


In [4]:
from hpecp import ContainerPlatformClient, APIException
from hpecp.k8s_cluster import K8sClusterHostConfig
import os, sys

hpeclient = ContainerPlatformClient(username='admin', 
                                password='admin123', 
                                api_host='127.0.0.1', 
                                api_port=8080,
                                use_ssl=True,
                                verify_ssl='/certs/hpecp-ca-cert.pem')

hpeclient.create_session()

In [5]:
print( hpeclient.k8s_worker.get_k8shosts().tabulate() )

+-----------+------------+------------------------------------------+------------+--------------------------+
| worker_id |   status   |                 hostname                 |   ipaddr   |           href           |
+-----------+------------+------------------------------------------+------------+--------------------------+
|     3     | configured | ip-10-1-0-145.eu-west-2.compute.internal | 10.1.0.145 | /api/v2/worker/k8shost/3 |
|     4     | configured | ip-10-1-0-50.eu-west-2.compute.internal  | 10.1.0.50  | /api/v2/worker/k8shost/4 |
|     5     | configured | ip-10-1-0-181.eu-west-2.compute.internal | 10.1.0.181 | /api/v2/worker/k8shost/5 |
|     6     | configured | ip-10-1-0-118.eu-west-2.compute.internal | 10.1.0.118 | /api/v2/worker/k8shost/6 |
+-----------+------------+------------------------------------------+------------+--------------------------+


In [6]:
try:
    k8s_cluster_id = hpeclient.k8s_cluster.create(
                                name = 'Demo Cluster 2', 
                                description = 'Demo 2 using Python', 
                                k8s_version = '1.17.0', 
                                k8shosts_config = [ 
                                                K8sClusterHostConfig('/api/v2/worker/k8shost/5', 'master')
                                            ]
                                )
    print('Creating cluster id: ' + k8s_cluster_id)
except APIException as e:
    print(e)

2020-05-01 10:37:11,498 - ContainerPlatformClient - ERROR - k8s_cluster/create : post https://127.0.0.1:8080/api/v2/k8scluster Request: {"label": {"name": "Demo Cluster 2", "description": "Demo 2 using Python"}, "pod_network_range": "10.192.0.0/12", "service_network_range": "10.96.0.0/12", "pod_dns_domain": "cluster.local", "persistent_storage": {"local": false, "nimble_csi": false}, "k8shosts_config": [{"node": "/api/v2/worker/k8shost/5", "role": "master"}], "k8s_version": "1.17.0"}
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/hpecp/client.py", line 152, in _request
    response.raise_for_status()
  File "/usr/local/lib/python3.7/site-packages/requests/models.py", line 941, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://127.0.0.1:8080/api/v2/k8scluster


('403 Forbidden: \n/api/v2/worker/k8shost/5.node: host is part of an existing k8s cluster\r\n', 'post', 'https://127.0.0.1:8080/api/v2/k8scluster', '{"label": {"name": "Demo Cluster 2", "description": "Demo 2 using Python"}, "pod_network_range": "10.192.0.0/12", "service_network_range": "10.96.0.0/12", "pod_dns_domain": "cluster.local", "persistent_storage": {"local": false, "nimble_csi": false}, "k8shosts_config": [{"node": "/api/v2/worker/k8shost/5", "role": "master"}], "k8s_version": "1.17.0"}')


In [7]:
print("\nHPE Container Platform K8S Clusters:\n")

print(hpeclient.k8s_cluster.list().tabulate())


HPE Container Platform K8S Clusters:

+-----------------------+----------------+---------------------+-------------+--------------------+----------------------+--------------+--------+
|          id           |      name      |     description     | k8s_version | created_by_user_id | created_by_user_name | created_time | status |
+-----------------------+----------------+---------------------+-------------+--------------------+----------------------+--------------+--------+
| /api/v2/k8scluster/46 |      Demo      |  Demo using Python  |   1.17.0    |   /api/v1/user/5   |        admin         |  1588313816  | ready  |
| /api/v2/k8scluster/47 |   Cluster 2    |   Created via UI    |   1.16.4    |   /api/v1/user/5   |        admin         |  1588313906  | ready  |
| /api/v2/k8scluster/48 | Demo Cluster 2 | Demo 2 using Python |   1.17.0    |   /api/v1/user/5   |        admin         |  1588314522  | ready  |
+-----------------------+----------------+---------------------+-------------+-

In [5]:
from hpecp import APIException, APIItemNotFoundException
try:
    hpeclient.k8s_cluster.wait(k8scluster_id='/api/v2/k8scluster/46', status=[ ])
except:
    print("Error")

ImportError: cannot import name 'APIItemNotFoundException' from 'hpecp' (/usr/local/lib/python3.7/site-packages/hpecp/__init__.py)

In [6]:
from kubernetes import client, config
import tempfile

try:
    demo_cluster = hpeclient.k8s_cluster.get('/api/v2/k8scluster/46')
except APIException as e:
    print(e)
    
# get the kube admin config for the first cluster 
admin_kube_config = demo_cluster.admin_kube_config

#  write it to a temporary file
with tempfile.NamedTemporaryFile(mode = "w") as fp:
    fp.write(admin_kube_config)
    fp.flush()

    # read the kube config with the kubernetes client
    config.load_kube_config(fp.name)

    # list the pods        
    v1 = client.CoreV1Api()
    print("\nListing pods with their IPs:\n")
    ret = v1.list_pod_for_all_namespaces(watch=False)
    for i in ret.items:
        print("{:>12}   {:>20}   {}".format(i.status.pod_ip, i.metadata.namespace, i.metadata.name))



Listing pods with their IPs:

10.192.0.132                  hpecp   hpecp-agent-8499d5798b-rh6mc
   10.1.0.50                  hpecp   hpecp-fsmount-bxnqp
  10.1.0.145                  hpecp   hpecp-fsmount-vlk4v
10.192.0.133                  hpecp   kubedirector-7fdbdb7775-9dzdb
  10.1.0.145            kube-system   canal-79glm
   10.1.0.50            kube-system   canal-vx8sh
10.192.0.131            kube-system   coredns-6955765f44-9vhmr
10.192.0.130            kube-system   coredns-6955765f44-dgws2
   10.1.0.50            kube-system   etcd-ip-10-1-0-50.eu-west-2.compute.internal
   10.1.0.50            kube-system   kube-apiserver-ip-10-1-0-50.eu-west-2.compute.internal
   10.1.0.50            kube-system   kube-controller-manager-ip-10-1-0-50.eu-west-2.compute.internal
  10.1.0.145            kube-system   kube-proxy-8q5sq
   10.1.0.50            kube-system   kube-proxy-d5v6r
   10.1.0.50            kube-system   kube-scheduler-ip-10-1-0-50.eu-west-2.compute.internal
10.192.0.13